# 01. NLP에서의 원-핫 인코딩(One-hot encoding)

In [1]:
from konlpy.tag import Okt  
okt = Okt()  
token = okt.morphs("나는 자연어 처리를 배운다")  
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [2]:
word2index = {}
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [3]:
def one_hot_encoding(word, word2index):
    one_hot_vector = [0]*(len(word2index))
    index = word2index[word]
    one_hot_vector[index] = 1
    return one_hot_vector

In [4]:
one_hot_encoding("자연어",word2index)

[0, 0, 1, 0, 0, 0]

단점
- 벡터를 저장하기 위해 필요한 공간이 계속 늘어난다
- 단어의 유사도를 표현하지 못한다

# 02. 워드 임베딩(Word Embedding)

## 1. 희소 표현(Sparse Representation)

In [5]:
import torch
# 원-핫 벡터 생성
dog = torch.FloatTensor([1, 0, 0, 0, 0])
cat = torch.FloatTensor([0, 1, 0, 0, 0])
computer = torch.FloatTensor([0, 0, 1, 0, 0])
netbook = torch.FloatTensor([0, 0, 0, 1, 0])
book = torch.FloatTensor([0, 0, 0, 0, 1])

In [6]:
print(torch.cosine_similarity(dog, cat, dim=0))
print(torch.cosine_similarity(cat, computer, dim=0))
print(torch.cosine_similarity(computer, netbook, dim=0))
print(torch.cosine_similarity(netbook, book, dim=0))

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


## 2. 워드 임베딩(Word Embedding)

밀집 표현(Dense Representation): Sparse Representation과 반대의 개념으로 0의 개수가 적은 것

워드 임베딩(word embedding): 단어를 밀집 벡터(dense vector)의 형태로 표현하는 방법, 임베딩 벡터(embedding vector)라고도 함

워드 임베딩 방법론
- LSA
- Word2Vec
- FastText
- Glove 
- 등
| - | 원-핫 벡터 | 임베딩 벡터 |
|:---:|---|---|
| - | 원-핫 벡터 | 임베딩 벡터 |
| 차원 | 고차원(단어 집합의 크기) | 저차원 |
| 다른 표현 | 희소 벡터의 일종 | 밀집 벡터의 일종 |
| 표현 방법 | 수동 | 훈련 데이터로부터 학습함 |
| 값의 타입 | 1과 0 | 실수 |


## 03. 워드투벡터(Word2Vec) 훈련시키기

### 1. 영어 Word2Vec 만들기

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/qwe1443/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

#### 1)훈련 데이터 다운로드(XML 파일)

In [18]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f18820ff7f0>)

#### 2)훈련 데이터 전처리하기

In [19]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [20]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [21]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


#### 3)Word2Vec 훈련시키기

In [22]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

- __size__ = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- __window__ = 컨텍스트 윈도우 크기
- __min_count__ = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- __workers__ = 학습을 위한 프로세스 수
- __sg__ = 0은 CBOW, 1은 Skip-gram.

In [23]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8445358276367188), ('guy', 0.8085939288139343), ('boy', 0.7854315042495728), ('lady', 0.7751310467720032), ('girl', 0.7562702894210815), ('soldier', 0.7297888994216919), ('kid', 0.7226467728614807), ('gentleman', 0.7207008004188538), ('surgeon', 0.676133394241333), ('son', 0.6623880863189697)]


#### 4) Word2Vec 모델 저장하고 로드하기

In [24]:
model.wv.save_word2vec_format('./eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

NameError: name 'KeyedVectors' is not defined

### 2. 한국어 Word2Vec 만들기

#### 1) 위키피디아 한국어 덤프 파일 다운로드
```bash
wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
```
#### 2) 위키피디아 익스트랙터 다운로드
```bash
git clone "https://github.com/attardi/wikiextractor.git"  
```
#### 3) 위키피디아 한국어 덤프 파일 변환
```bash
cd wikiextractor/text
python WikiExtractor.py kowiki-latest-pages-articles.xml.bz2 
```
#### 4) 훈련 데이터 만들기
```bash
cat AA/wiki* > wikiAA.txt
cat AB/wiki* > wikiAB.txt
cat AC/wiki* > wikiAC.txt
cat AD/wiki* > wikiAD.txt
cat AE/wiki* > wikiAE.txt
cat AF/wiki* > wikiAF.txt
cat AG/wiki* > wikiAG.txt
cat wikiA* > ~/wiki_data.txt
```
#### 5) 훈련 데이터 전처리 하기

In [27]:
f = open('wiki_data.txt', encoding="utf8")
i=0
while True:
    line = f.readline()
    if line != '\n':
        i=i+1
        print("%d번째 줄 :"%i + line)
    if i==5:
        break 
f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.

4번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

5번째 줄 :1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.



In [28]:
from konlpy.tag import Okt  
okt=Okt()
fread = open('wiki_data.txt', encoding="utf8")
# 파일을 다시 처음부터 읽음.
n=0
result = []

while True:
    line = fread.readline() #한 줄씩 읽음.
    if not line: break # 모두 읽으면 while문 종료.
    n=n+1
    if n%5000==0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 While문."%n)
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp) # 결과에 저장
fread.close()

5000번째 While문.
10000번째 While문.
15000번째 While문.
20000번째 While문.
25000번째 While문.
30000번째 While문.
35000번째 While문.
40000번째 While문.
45000번째 While문.
50000번째 While문.
55000번째 While문.
60000번째 While문.
65000번째 While문.
70000번째 While문.
75000번째 While문.
80000번째 While문.
85000번째 While문.
90000번째 While문.
95000번째 While문.
100000번째 While문.
105000번째 While문.
110000번째 While문.
115000번째 While문.
120000번째 While문.
125000번째 While문.
130000번째 While문.
135000번째 While문.
140000번째 While문.
145000번째 While문.
150000번째 While문.
155000번째 While문.
160000번째 While문.
165000번째 While문.
170000번째 While문.
175000번째 While문.
180000번째 While문.
185000번째 While문.
190000번째 While문.
195000번째 While문.
200000번째 While문.
205000번째 While문.
210000번째 While문.
215000번째 While문.
220000번째 While문.
225000번째 While문.
230000번째 While문.
235000번째 While문.
240000번째 While문.
245000번째 While문.
250000번째 While문.
255000번째 While문.
260000번째 While문.
265000번째 While문.
270000번째 While문.
275000번째 While문.
280000번째 While문.
285000번째 While문.
290000번째 While문.
295000번째 While문.
300000번째 While문.


java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: Java heap space: failed reallocation of scalar replaced objects

#### 6) Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model_result1=model.wv.most_similar("대한민국")
print(model_result1)

In [ ]:
model_result2=model.wv.most_similar("어벤져스")
print(model_result2)

In [ ]:
model_result3=model.wv.most_similar("반도체")
print(model_result3)